In [75]:
import pandas
import string
import nltk
import keras
import sklearn
import time
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy
import tensorflow as tf
import numpy as np
from time import time
from nltk.corpus import stopwords
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.callbacks import TensorBoard
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense , Activation , Dropout
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
import keras.preprocessing.text

In [76]:
Train_X = pandas.read_csv("../Dataset/cleaned_train.csv", header=0)['text']
Test_X = pandas.read_csv("../Dataset/cleaned_dev.csv", header=0)['text']

Train_Y_stance= pandas.read_csv("../Dataset/cleaned_train.csv", header=0)['stance']
Train_Y_category= pandas.read_csv("../Dataset/cleaned_train.csv", header=0)['category']

Test_Y_stance= pandas.read_csv("../Dataset/cleaned_dev.csv", header=0)['stance']
Test_Y_category= pandas.read_csv("../Dataset/cleaned_dev.csv", header=0)['category']

In [77]:
# concatenate both dev and train texts
texts = Train_X.append(Test_X, ignore_index=True)


tokenizer = Tokenizer(num_words=None,lower=False)
tokenizer.fit_on_texts(texts)

Train_X = tokenizer.texts_to_matrix(Train_X, mode='tfidf')
Test_X = tokenizer.texts_to_matrix(Test_X, mode='tfidf')

C:\Users\hazem\AppData\Local\Temp\ipykernel_16200\1537059493.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  texts = Train_X.append(Test_X, ignore_index=True)


In [78]:
# concatenate the stance labels in dev and train
stances = Train_Y_stance.append(Test_Y_stance, ignore_index=True)

# encoding my labels from text to numbers not necessary for stance
encoder = LabelEncoder()
encoder.fit(stances)
stancess=encoder.fit_transform(stances)

num_classes = int((len(set(stancess))))
print((len(set(stancess))))

# tranforming my labels to their encoded numbers
Train_Y_stance = encoder.fit_transform(Train_Y_stance)
Test_Y_stance = encoder.fit_transform(Test_Y_stance)

3


C:\Users\hazem\AppData\Local\Temp\ipykernel_16200\1391908612.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stances = Train_Y_stance.append(Test_Y_stance, ignore_index=True)


In [79]:
Train_Y_stance= keras.utils.to_categorical(Train_Y_stance,num_classes)
Test_Y_stance = keras.utils.to_categorical(Test_Y_stance, num_classes)


num_labels = int(len(Train_Y_stance.shape))
vocab_size = len(tokenizer.word_index) + 1

max_words=vocab_size

In [80]:
import keras.backend as K
def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

from keras.metrics import Precision , Recall , Accuracy , TruePositives , TrueNegatives , FalsePositives , FalseNegatives

In [81]:
# Build the model
model = Sequential()
model.add(Dense(1024, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [82]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy','Recall','Precision', f1_metric,'TruePositives','TrueNegatives','FalsePositives','FalseNegatives'])

In [83]:
batch_size = 50
epochs = 3

history = model.fit(Train_X, Train_Y_stance,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)


model.save('my_model.h1')

Epoch 1/3
126/126 [==============================] - 77s 604ms/step - loss: 0.6064 - categorical_accuracy: 0.7892 - recall: 0.8806 - precision: 0.6518 - f1_metric: 0.7519 - true_positives: 5538.0000 - true_negatives: 9619.0000 - false_positives: 2959.0000 - false_negatives: 751.0000 - val_loss: 0.5310 - val_categorical_accuracy: 0.7969 - val_recall: 0.9270 - val_precision: 0.6000 - val_f1_metric: 0.7290 - val_true_positives: 648.0000 - val_true_negatives: 966.0000 - val_false_positives: 432.0000 - val_false_negatives: 51.0000
Epoch 2/3
126/126 [==============================] - 75s 595ms/step - loss: 0.1503 - categorical_accuracy: 0.9580 - recall: 0.9892 - precision: 0.7848 - f1_metric: 0.8760 - true_positives: 6221.0000 - true_negatives: 10872.0000 - false_positives: 1706.0000 - false_negatives: 68.0000 - val_loss: 0.6808 - val_categorical_accuracy: 0.8054 - val_recall: 0.9013 - val_precision: 0.6695 - val_f1_metric: 0.7688 - val_true_positives: 630.0000 - val_true_negatives: 1087.000

INFO:tensorflow:Assets written to: my_model.h1\assets


INFO:tensorflow:Assets written to: my_model.h1\assets


In [84]:
#model = keras.models.load_model('my_model.h1')
Evaluation_valus = model.evaluate(Test_X,Test_Y_stance,verbose=0)
print("Loss" , 'categorical_accuracy','Recall','Precision','f1_metric','TruePositives','TrueNegatives','FalsePositives','FalseNegatives')

print(Evaluation_valus)

Loss categorical_accuracy Recall Precision f1_metric TruePositives TrueNegatives FalsePositives FalseNegatives
[0.7929034233093262, 0.8009999990463257, 0.890999972820282, 0.689095139503479, 0.779004693031311, 891.0, 1598.0, 402.0, 109.0]


In [85]:
import csv

Test_X = pandas.read_csv("../Dataset/cleaned_dev.csv", header=0)['text']
Test_Y_stance = pandas.read_csv("../Dataset/cleaned_dev.csv", header=0)['stance']

In [86]:
fields = ['Text', 'Predicted', 'Actual','isCorrect']

filename = "../Results/AraBert.csv"

correct_count = 0
predictions=[]

with open(filename, 'w',encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)

    for i in range(len(Test_X)):
        tokens = tokenizer.texts_to_matrix([Test_X[i]], mode='tfidf')
        predictions_vector =model.predict(np.array(tokens))
        predicted_class_encoded =np.argmax(predictions_vector,axis=1)
        predicted_class_name = encoder.inverse_transform(predicted_class_encoded)[0]

        isCorrect = "False"
        predictions.append(predicted_class_name)
        if predicted_class_name == Test_Y_stance[i]:
            correct_count += 1
            isCorrect = "True"
        
        csvwriter.writerow([Test_X[i], str(predicted_class_name), str(Test_Y_stance[i]), isCorrect])

1/1 [==============================] - 0s 38ms/step


In [87]:
from sklearn.metrics import f1_score

accuracy = correct_count / len(Test_X)
print("F1 score ", f1_score(Test_Y_stance, predictions, average='macro'))
print("Accuracy: ", accuracy)


F1 score  0.4884829731004201
Accuracy:  0.801
